Вы аналитик российской авиакомпании F9, выполняющей внутренние пассажирские перевозки. Важно понять предпочтения пользователей, покупающих билеты на разные направления.

Вам предстоит изучить базу данных и проанализировать спрос пассажиров на рейсы в города, где проходят крупнейшие культурные фестивали.

**Аналитика средствами Python**

У вас есть файлы, в которых содержатся результаты запросов из предыдущих заданий.

**query_1.csv** — результат первого запроса. В нём содержится такая информация:

- *model* — **модель самолета;
- *flights_amount* — количество рейсов для каждой модели самолетов *model* в сентябре 2018 года.

**query_3.csv** — результат третьего запроса. В нём содержится такая информация:

- *city* — город;
- *average_flights* — среднее количество рейсов, прибывающих в город (*city*) за день в сентябре 2018 года.

 Для этих двух наборов данных нужно:

- импортировать файлы;
- изучить данные в них;
- проверить типы данных на корректность;
- выбрать топ-10 городов по количеству рейсов;
- построить графики: модели самолётов и количество рейсов, города и количество рейсов, топ-10 городов и количество рейсов;
- сделать выводы по каждому из графиков, пояснить результат.


# Аналитика

## Импорт и изучение данных

In [1]:
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px
import plotly.graph_objects as go

In [2]:
!pip install plotly==4.14.3 --quiet

In [3]:
df_model = pd.read_csv('/datasets/query_1.csv')
df_city = pd.read_csv('/datasets/query_3.csv')

In [4]:
df_model

,model,flights_amount
0,Airbus A319-100,607
1,Airbus A321-200,960
2,Boeing 737-300,630
3,Boeing 767-300,600
4,Boeing 777-300,300
5,Bombardier CRJ-200,4446
6,Cessna 208 Caravan,4557
7,Sukhoi SuperJet-100,4185


In [5]:
df_city

,city,average_flights
0,Абакан,3.870968
1,Анадырь,1.000000
2,Анапа,2.161290
3,Архангельск,5.354839
4,Астрахань,2.451613
...,...,...
96,Чита,1.580645
97,Элиста,4.870968
98,Южно-Сахалинск,4.290323
99,Якутск,2.741935


In [6]:
def getInfo(df):
    print(df.info())
    print(df.describe())
    print((len(df) - df.nunique()))

In [7]:
getInfo(df_model)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
model             8 non-null object
flights_amount    8 non-null int64
dtypes: int64(1), object(1)
memory usage: 256.0+ bytes
None
       flights_amount
count        8.000000
mean      2035.625000
std       1965.203947
min        300.000000
25%        605.250000
50%        795.000000
75%       4250.250000
max       4557.000000
model             0
flights_amount    0
dtype: int64


In [8]:
getInfo(df_city)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
city               101 non-null object
average_flights    101 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB
None
       average_flights
count       101.000000
mean          5.494189
std          13.119790
min           1.000000
25%           2.000000
50%           3.000000
75%           5.870968
max         129.774194
city                0
average_flights    41
dtype: int64


### Вывод
<p>Дубликатов моделей и городов не обнаружено. Все типы данных корректны.</p>
<p>Разброс количества полетов в зависимости от моделей большой - от 300 до 4500 тысяч. Скорей всего это связано с количеством посадочных мест для каждого самолета.</p>
<p>Среднее количество полетов в города имеет также большой разброс - от 1 до 130. Выдвинем гипотезу, что на N количества городов приходится больше 50% перелетов.</p>

## Топ-10 городов по количеству рейсов

In [9]:
top10 = df_city.sort_values(by='average_flights', ascending=False).head(10)

In [10]:
top10

,city,average_flights
43,Москва,129.774194
70,Санкт-Петербург,31.161290
54,Новосибирск,17.322581
33,Красноярск,11.580645
20,Екатеринбург,11.322581
67,Ростов-на-Дону,10.193548
63,Пермь,10.129032
10,Брянск,10.000000
74,Сочи,9.612903
84,Ульяновск,9.580645


In [11]:
fig = px.pie(top10, values='average_flights', names='city', title='Топ-10 городов по количеству рейсов')
fig.show()

Расчитаем, какую долю перелетов принимают на себя топ-10 городов.

In [12]:
print('На топ 10 городов процент перелетов составляет:',top10['average_flights'].sum()/df_city['average_flights'].sum()*100)

На топ 10 городов процент перелетов составляет: 45.17417576330091


In [13]:
print('На Москву процент перелетов составляет:',top10[top10['city'] == 'Москва']['average_flights'].sum()/df_city['average_flights'].sum()*100)

На Москву процент перелетов составляет: 23.386399317431415


### Вывод
<p>Почти половина рейсов приходится на топ-10 городов. На Москву приходится 25% перелетов. Следом идут города миллионеры -  Санкт-Петербург, Новосибирск, Красноярск, Ектаерингбур и Ростов.</p>

## График моделей самолётов и количество рейсов

In [14]:
fig = px.bar(df_model.sort_values(by='flights_amount', ascending=False), x='model', y='flights_amount', title='Модели самолётов и количество рейсов')
fig.show()

На основе данных https://www.airlines-inform.ru/ возьмем инфомрацию о количестве мест в самолете в зависимости от модели и посчитаем общий пассажиропоток в сентрябре 2018 года.  

In [15]:
places = [116,185,149,328,550,50,13,98]
df_model['places'] = places

In [16]:
df_model['passengers'] = df_model['flights_amount']*df_model['places']
df_model

,model,flights_amount,places,passengers
0,Airbus A319-100,607,116,70412
1,Airbus A321-200,960,185,177600
2,Boeing 737-300,630,149,93870
3,Boeing 767-300,600,328,196800
4,Boeing 777-300,300,550,165000
5,Bombardier CRJ-200,4446,50,222300
6,Cessna 208 Caravan,4557,13,59241
7,Sukhoi SuperJet-100,4185,98,410130


In [17]:
fig = px.bar(df_model.sort_values(by='passengers', ascending=False), x='model', y='passengers', title='Количество пассажиров')
fig.show()

### Вывод
<p>Большинство рейсов совершали самолеты с количеством мест для пассажиров менее 100. Именно поэтому разница полетов для самолетов разной модели велика.</p>
<p>Если рассчитать количество пассажиров, которые перевезли самолеты в сентябре 2018, то тут картина немного поменяется. Например самолет Cessna 208 Caravan совершил больше всех полетов, но перевез меньше всего людей, а вот Boeing 777 с последнего места по полетам перешел на 5 позицию по пассажиропотоку.</p>

## График городов и количество рейсов

Поскольку нам нужно построить график с количеством рейсов, а не их средним показателями в день, то рассчитаем новый столбец.

In [18]:
df_city['flights']=df_city['average_flights']*30
df_city['flights'] = df_city['flights'].round(0)
df_city

,city,average_flights,flights
0,Абакан,3.870968,116.0
1,Анадырь,1.000000,30.0
2,Анапа,2.161290,65.0
3,Архангельск,5.354839,161.0
4,Астрахань,2.451613,74.0
...,...,...,...
96,Чита,1.580645,47.0
97,Элиста,4.870968,146.0
98,Южно-Сахалинск,4.290323,129.0
99,Якутск,2.741935,82.0


### Вывод
<p>По графику можно сделать вывод, что чем больше численность города, тем больше рейсов в него совершают самолеты. Данный факт вполне очевиден. Стоит также отметить, что в топе находится город Сочи. Скорей всего его присутсвие на данной позиции охарактеризовано тем, что в этот период он активно пользуется спросом у туристов. </p>

## Общий вывод

Во время проекта:

1) изучили и подготовили данные:

    посмотрели общую и статистическую информацию о данных
    убедились, что пропуски и дубликаты отсутствуют
    проверили тип данных на корректность

2) провели анализ данных и построили графики:

    модели самолетов и количество рейсов
    топ-10 городов и количество рейсов
    города и количество рейсов

После проведенного анализа можно сделать следующие выводы:

- Больше всего рейсов в сентябре 2018 года было совершено на самолетах таких моделей как Cessna 208 Caravan, Bombardier CRJ-200 и Sukhoi Superjet-100. Меньше всего рейсов было совершено на самолетах модели Boeing 777-300 (300 рейсов).

- Как топ-3 города с наибольшим средним количеством рейсов можно выделить: Москва, Санкт-Петербург и Новосибирск.

- Реже всего рейсы отправляются в маленькие города: Иваново, Ярославль, Братск и прочее. В среднем за день в эти города прибыл только один рейс.


